In [1]:
import numpy as np
import copy
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from keras import Sequential, Model, layers, callbacks, optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam


2024-05-25 12:58:57.018186: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 12:58:57.018246: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 12:58:57.019802: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-25 12:58:57.028507: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
milk_df = pd.read_csv("milk.csv", parse_dates=["Date"])
milk_df.dropna(how='any', axis=0, inplace=True, ignore_index=True)
df = milk_df.drop('Date', axis=1)
df = df.drop('dim2', axis=1)
df = df.drop('Cond', axis=1)
df = df.drop('Peak', axis=1)
print(df)

       AnEar  lactation  dim  milk_production  MilkShif    Dur
0        123          4    1             11.5     11.50  276.0
1        123          4    2             48.4     24.20  384.0
2        123          4    3             66.4     33.20  672.0
3        123          4    4             57.1     28.55  354.0
4        123          4    5             78.5     39.25  396.0
...      ...        ...  ...              ...       ...    ...
25841   9977          2  301             72.8     36.40  540.0
25842   9977          2  302             73.0     36.50  600.0
25843   9977          2  303             70.9     35.45  540.0
25844   9977          2  304             70.6     35.30  516.0
25845   9977          2  305             67.7     33.85  522.0

[25846 rows x 6 columns]


In [3]:
dv_treino = {}
dv_teste = {}
nomes_vacas = df['AnEar'].unique()
print(nomes_vacas)
for nome in nomes_vacas:
    vals = df[df['AnEar'] == nome].values
    vals = np.delete(vals, 0, axis=1)
    dv_treino[nome] = vals[:50]
    dv_teste[nome] = vals[50:]


[ 123 1009 1017 1209 1218 1242 1243 1275 1280 1305 1327 1329 1341 1362
 1368 1387 1393 7761 8064 8334 8385 8393 8434 8435 8438 8509 8581 8667
 8723 8732 8741 8932 8947 8996 9005 9066 9069 9078 9081 9090 9101 9149
 9188 9207 9284 9346 9363 9388 9389 9431 9463 9464 9478 9487 9488 9525
 9529 9549 9558 9562 9583 9885 9891 9892 9894 9908 9909 9910 9917 9918
 9920 9922 9926 9928 9929 9933 9939 9943 9944 9946 9951 9961 9967 9968
 9977]


In [4]:
def get_model(input_size, hidden_layer_sizes, dropout, learning_rate, loss):
  model=Sequential()
  model.add(layers.InputLayer((input_size,)))
  model.add(layers.Flatten())
  model.add(layers.Dense(hidden_layer_sizes, activation="relu", kernel_initializer="he_uniform"))
  model.add(layers.Dropout(dropout))
  model.add(layers.BatchNormalization())
  model.add(layers.Dense(hidden_layer_sizes, activation="relu", kernel_initializer="he_uniform"))
  model.add(layers.Dropout(dropout))
  model.add(layers.BatchNormalization())
  model.add(layers.Dense(hidden_layer_sizes, activation="relu", kernel_initializer="he_uniform"))
  model.add(layers.Dropout(dropout))
  model.add(layers.BatchNormalization())
  model.add(layers.Dense(hidden_layer_sizes, activation="relu", kernel_initializer="he_uniform"))
  model.add(layers.Dropout(dropout))
  model.add(layers.Dense(1,activation="linear"))

  #final_model = Model([model.input], out)
  model.compile(loss=loss, optimizer=optimizers.Adam(learning_rate=learning_rate), metrics=['mae'])

  return model

In [5]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-05-25 12:58:59.334356: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-25 12:58:59.346589: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-25 12:58:59.349957: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
all_values = np.concatenate(list(dv_treino.values()), axis=0)
x_t = np.delete(all_values, 2, axis=1)
y_t = all_values[:, 2].reshape(-1, 1)

x_mmc = MinMaxScaler().fit(x_t)
X = x_mmc.transform(x_t)

y_mmc = MinMaxScaler().fit(y_t)
Y = y_mmc.transform(y_t)

all_test_values = np.concatenate(list(dv_teste.values()), axis=0)
x_test = np.delete(all_test_values, 2, axis=1)
y_test = all_test_values[:, 2].reshape(-1, 1)

X_final = x_mmc.transform(x_test)
Y_final = y_mmc.transform(y_test)

In [7]:
cols = [
    "tipo",
    "hidden_layer_sizes", 
    "learnRate",
    "dropout",
    "batchSize",
    "fold",
    "MAE",
    ]

kf = KFold(n_splits=3, shuffle=False)
i=0
for hls in [256, 512, 784]:
    for lr in [0.01, 0.001, 0.0001]:
        for do in [0.3, 0.4, 0.5]:
            for bs in [16, 32]:
                j=0
                for train_index, val_index in kf.split(X):
                    model = get_model(input_size=4, hidden_layer_sizes=hls, dropout=do, learning_rate=lr, loss='mean_absolute_error')
                    x_treino, x_valid = X[train_index], X[val_index]
                    y_treino, y_valid = Y[train_index], Y[val_index]
                    name = f"SEM_{i}"
                    hist = model.fit(
                        x_treino,
                        y_treino,
                        batch_size=bs,
                        epochs=50,
                        validation_data=(x_valid, y_valid),
                        callbacks=[
                            ModelCheckpoint(filepath=f"ps/{name}.keras",save_best_only=True,verbose=0)
                        ],
                        verbose=0
                    )
                    pred = model.predict(X_final, verbose=0)
                    vals = [[
                        "SEM",
                        hls,
                        lr,
                        do,
                        bs,
                        j,
                        mean_absolute_error(Y_final, pred)
                    ]]
                    pd.DataFrame(vals, columns=cols).to_csv(f'ps/{name}.csv', index=False)
                    print(f"FIM - {i}/161  - SEM - {hls} - {lr} - {do} - {bs} - {j}")
                    i+=1
                    j+=1

2024-05-25 12:58:59.390363: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-25 12:58:59.395817: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-25 12:58:59.398907: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

FIM - 0/161  - SEM - 256 - 0.01 - 0.3 - 16 - 0
FIM - 1/161  - SEM - 256 - 0.01 - 0.3 - 16 - 1
FIM - 2/161  - SEM - 256 - 0.01 - 0.3 - 16 - 2
FIM - 3/161  - SEM - 256 - 0.01 - 0.3 - 32 - 0
FIM - 4/161  - SEM - 256 - 0.01 - 0.3 - 32 - 1
FIM - 5/161  - SEM - 256 - 0.01 - 0.3 - 32 - 2
FIM - 6/161  - SEM - 256 - 0.01 - 0.4 - 16 - 0
FIM - 7/161  - SEM - 256 - 0.01 - 0.4 - 16 - 1
FIM - 8/161  - SEM - 256 - 0.01 - 0.4 - 16 - 2
FIM - 9/161  - SEM - 256 - 0.01 - 0.4 - 32 - 0
FIM - 10/161  - SEM - 256 - 0.01 - 0.4 - 32 - 1
FIM - 11/161  - SEM - 256 - 0.01 - 0.4 - 32 - 2
FIM - 12/161  - SEM - 256 - 0.01 - 0.5 - 16 - 0
FIM - 13/161  - SEM - 256 - 0.01 - 0.5 - 16 - 1
FIM - 14/161  - SEM - 256 - 0.01 - 0.5 - 16 - 2
FIM - 15/161  - SEM - 256 - 0.01 - 0.5 - 32 - 0
FIM - 16/161  - SEM - 256 - 0.01 - 0.5 - 32 - 1
FIM - 17/161  - SEM - 256 - 0.01 - 0.5 - 32 - 2
FIM - 18/161  - SEM - 256 - 0.001 - 0.3 - 16 - 0
FIM - 19/161  - SEM - 256 - 0.001 - 0.3 - 16 - 1
FIM - 20/161  - SEM - 256 - 0.001 - 0.3 - 16 - 2